In [ ]:
# Burada ilk önce modeli yükleyeceğiz.
# O yüzden dosya dizinini modelin bulunduğu klasör olarak değiştiriyoruz.
%cd yolov7

In [ ]:
# Ardından modeli yüklüyoruz. 
# Burada önceden eğitilmiş olan ağırlıkları kullandım.
# Burada kullanacağımız ağrıkların adını yazıyoruz.
# böylece modeli tanımlamış oluyoruz.
# Bu arada Pytorch kurulu olmalı bilgisayrınızda ayrıca cuda hali kurulu olmalı.
# yoksa hata alırsınız.
from  hubconf import custom
model = custom(path_or_model='yolov7.pt')  # custom example
# model = create(name='yolov7', pretrained=True, channels=3, classes=80, autoshape=True)  # pretrained example

In [ ]:
# Burada bir dizini bir üst dizin olarak değiştiriyıruz.
# Böylece kod dosyasının bulunduğu ana dizine geldik
%cd ..

In [ ]:
# Bundan sonra ise kullanacağımız tracker objesini tanımlayacağız. 
# Burada deep sort kullanıyoruz. O yüzden o dizinin içine giriyoruz.



%cd deep_sort

In [ ]:
# burada gerekli objeleri import ediyoruz.

from deep_sort import  nn_matching
from application_util import preprocessing
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker
from tools import generate_detections as gdet

In [ ]:
# burada tracker objesindeki bazı parametreleri ayarlıyoruz.
max_cosine_distance = 0.4
nn_budget = None

# Burada tracker objesini tanımlıyoruz. Bunu kullanarak her bir nesneye bir id atıyrouz.

model_filename = 'mars-small128.pb'
encoder = gdet.create_box_encoder(model_filename, batch_size=1)
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
tracker = Tracker(metric)

In [ ]:
# Tekrardan ana repo'ya dönüyoruz. 

%cd ..

In [ ]:
# Kütüphaneleri yüklüyoruz.

import cv2
import time
import numpy as np

In [ ]:
# Eğer ayrı bir environment oluşturduysanız bunu çalıştırmanıza gerek olmayabilir.
# fakat ana envirionment'ı kullanıyırsanız bunu çalıştırmazsanız tracking kısmında hata alabilirsiniz.

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
# burada etiketleri tanımlıyoruz.


labels=[ 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
         'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
         'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
         'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
         'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
         'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
         'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
         'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
         'hair drier', 'toothbrush' ]

In [ ]:
# burada kullanacağımız videonun adını giriyoruz.
kamera= cv2.VideoCapture('video1.mp4')
# Yazınını fontunu belirliyoruz.
font = cv2.FONT_HERSHEY_SIMPLEX

# Burada yolun sol tarafı ve sağ tarafı için bölgeler belirliyoruz.
# Araçlar bu bölgeye girdiği zaman sayılmaya başlanacak.
# Direk çizgi kullanamıyoruz. Çünkü biz burada araçların spesifik bir noktasına bakarak geçip geçmediğine karar veriyoruz.
# O nokta ile çizgi kesişmeyebilir. Bu yüzden dikey genişliği çok küçük olan bölgeler kullanıyoruz.
# Eğer aracın istenen noktası o bölgeye girdiyse o aracın id'sini hangi bölgeden geçtiğine göre bölgeye ekliyoruz.

# Ben bu bölgeleri kullandığım videoya göre belirledim.
# Bu bölgeler videoya göre değişir. 

# Burası yolun sol tarafı için
region1=np.array([(300,500),(650,500),(650,520),(200,520)])
region1 = region1.reshape((-1,1,2))


# # Burası yolun sağ tarafı için
region2=np.array([(700,500),(1200,500),(1200,480),(700,480)])
region2 = region2.reshape((-1,1,2))


# O bölgeye giren araçların id'lerini buradaki kümelerin içine ekliyoruz.
# Kümeden her bir elemandan sadece bir tane olabilir.
# O yüzden araç o bölgeye birden fazla girse bile sadece bir kere sayacaktır.

# Bu yolun sol tarafı için
toplam_id_sol=set()
# Bu yolun sağ tarafı için
toplam_id_sag=set()
count=0
while True:


    ret,kare=kamera.read()

    # resmi RGB'ye çeviridk.
    imgs=cv2.cvtColor(kare,cv2.COLOR_BGR2RGB)
    
    # Burada giriş bölgeleri yorumda.
    # Yani ekrana çizdirmedim.
    # Siz isterseniz çizdirebilirsiniz.

    #cv2.polylines(kare,[region1],True,(0,0,255),2)
    #cv2.polylines(kare,[region2],True,(0,0,255),2)
    # Bunun yerine ekrana bir çizgi çizgirdim
    # Bu çizgi algoritmada bir işe yaramıyor. 
    # Sadece arabaların nerede sayılmaya başlandığını göstermek için var 
    cv2.line(kare, (200,500), (1200,500), (102,0,153), 2)
    
    # resmi modele verip sonuçları elde ettik.
    results = model(imgs) 
    
    bboxes=[]
    scores=[]
    names=[]
    # Resimdeki nesne sayısı kadar dönüyor döngü
    for i in range(len(results.xywh[0])):
        xmin,ymin,xmax,ymax,score,name=results.xyxy[0][i]
        xmin,ymin,xmax,ymax=int(xmin),int(ymin),int(xmax),int(ymax)
        x1,y1=xmin,ymin
        w1=xmax-xmin
        h1=ymax-ymin
       
        # Eğer nesnenin score'u 0.7'den küçükse onu almıyoruz.
        if score <0.7:
            continue
        
        bboxes.append([x1,y1,w1,h1])
        scores.append(score)
        names.append(name)
  
        
    # Bulunan sonuçları tracking işlemi için uygun formata çeviriyoruz.
    features = encoder(imgs, bboxes)

    detections = [Detection(bbox, score, feature) for bbox, score, feature in zip(bboxes, scores, features)]


    tracker.predict()
    # ilk frame'e bir işlem uygulamyoruz.
    # O yüzden böyle if bloğu var.
    if count==0:
        count+=1
        continue
    # Tracker'a sonuçları vererek tracking işlemi yapıyrouz. 
    tracker.update(detections)
    for track in tracker.tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue 

        # Burada nesnelerin konumunu elde ediyoruz. Tracking sonras
        bbox = track.to_tlwh()
        x,y,w,h=bbox
        x,y,w,h=int(x),int(y),int(w),int(h)
        # araçların ortasının koordinatlarını buluyoruz.
        cx=int(x+w/2)
        cy=int(y+h/2)
    
        # Nesnelerin (arabların) id'sini bulup bunu ids adlı bir değişkene kaydediyoruz.
        ids=track.track_id
        
        # Burada araçların o bölgeden geçip geçmediğine bakacağız
        # Burada sol taraf için bakıyoruz.
        # Aracın orta alt noktası için yapıyrouz bunu
        inside_region1=cv2.pointPolygonTest(region1,(cx,y+h),False)
        # Eğer araç soldaki bölgeye girmişse burası True değeri döner ve if bloğunun içine girilir.
        if inside_region1>0:
            #soldaki bölgeye giren aracın id'sini bu kümenin ekliyoruz
            toplam_id_sol.add(ids)
        # kümenin uzunluğu ne kadar ise toplam o kadar araç geçmiştir.
        
        
        toplam1='Gelen:'+str(len(toplam_id_sol))
        
        # Burada sol taraf için bakıyoruz.
        # Aracın sol üst noktası için yapıyrouz bunu
        inside_region2=cv2.pointPolygonTest(region2,(x,y),False)
        # Eğer araç sağdaki bölgeye girmişse burası True değeri döner ve if bloğunun içine girilir.
        if inside_region2>0:
            #sağdaki bölgeye giren aracın id'sini bu kümenin ekliyoruz
            toplam_id_sag.add(ids)
            
        # kümenin uzunluğu ne kadar ise toplam o kadar araç geçmiştir.
        toplam2='Giden:'+str(len(toplam_id_sag))
        
                       
        

        cv2.putText(kare, toplam1,(50, 50), font, 1.5, (0,200,255), 2)
        cv2.putText(kare, toplam2,(850, 50), font, 1.5, (0,0,255), 2)

    cv2.imshow("kamera",kare)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

kamera.release()

cv2.destroyAllWindows()